In order to successfully complete this assignment you need to participate both individually and in groups during class.  Have one of the instructors check your notebook and sign you out before leaving class. Turn in your assignment using D2L. 

---

# ICA 32: MPI Reduce Example

<img src="https://upload.wikimedia.org/wikipedia/commons/2/2a/Pi-unrolled-720.gif">
<p style="text-align: right;">Animation from: [Wikipedia](https://en.wikipedia.org/wiki/File:Pi-unrolled-720.gif)</p>

### Agenda for today's class (70 minutes)

</p>

2. (10 minutes) Pre-class Review 
1. (60 minutes) Pi Monte Carlo

---
# 1. Pre-class Review 


**QUESTION:** From a parallel point of view. What are some fundamental differences between the pi estimation programs we have looked at this semester?  How do these differences influence which parallel library we can use?

Based on if we are doing Monte Carlo simulations or integration estimation to approximate pi we can use different parallel libraries. Although CUDA works for both, MPI is better for the Monte Carlo simulatons and OpenMP is better for integration estimation

---

# 2. Pi Monte Carlo


Compare your notes/solutions from the pre-class and rewrite the following program as an MPI program.  **NOTE:** There are probablly hundreds of MPI solutions on-line for this problem. Lets avoid looking at the answers and see if we came come up with a resonable solution on our own.

&#9989; **<font color=red>DO THIS:</font>** Step 1: Write and test a makefile and submission script.

&#9989; **<font color=red>DO THIS:</font>** Step 2: Initialize and finalize MPI.

&#9989; **<font color=red>DO THIS:</font>** Step 3: Separate the Master (Rank 0) with the workers.

&#9989; **<font color=red>DO THIS:</font>** Step 4: Benchmark the results.


```c++

#include <stdlib.h>
#include <stdio.h>
#include <mpi.h>

#define sqr(x) ((x)*(x))
long random(void);

double dboard(int darts) {
    double x_coord, y_coord, pi, r;
    int score = 0;
    long rd;
    unsigned long cconst = RAND_MAX;

    for (int n = 0; n < darts; n++) {
        rd = random();
        r = (double)rd / cconst;
        x_coord = (2.0 * r) - 1.0;
        r = (double)random() / cconst;
        y_coord = (2.0 * r) - 1.0;

        if ((sqr(x_coord) + sqr(y_coord)) <= 1.0)
            score++;
    }

    pi = 4.0 * (double)score / (double)darts;
    return pi;
}

int main(int argc, char **argv) {
    int rank, size;
    int totalDarts = 1000000;
    int dartsPerProc;
    double localPi, globalPi;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    dartsPerProc = totalDarts / size;

    /* Seed each process differently */
    srandom(rank * 123456 + time(NULL));

    localPi = dboard(dartsPerProc);

    /* Reduce (average) all partial estimates */
    MPI_Reduce(&localPi, &globalPi, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        globalPi /= size;
        printf("Estimated PI = %.16f\n", globalPi);
    }

    MPI_Finalize();
    return 0;
}
 ```

-----
### Congratulations, we're done!

Have one of the instructors check your notebook and sign you out before leaving class. Turn in your assignment using D2L.

Written by Dr. Dirk Colbry, Michigan State University (Updated by Dr. Nathan Haut in Spring 2025)
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

----